## Laboratorio 07: Hunting with Data Science 
Alexa Bravo  18831  
Mayo 02, 2022

### PARTE 1 - Filtrado y Preprocesamiento

In [1]:
#Importamos las librerias 
import pandas as pd
import numpy as np
import json
import joblib
from joblib import load
import whois
from features import domain_tld_extract
from modelo_compara.caracteristicasDerivadas import H_entropy, proporcionVocalesConsonantes, posicionPrimerDigito

In [2]:
#Cargamos los datos. 
file = open('large_eve.json')
datos = file.readlines()
file.close()

#Volvemos un diccionario el json, para poder obtener la cantidad total. 
registros = [json.loads(fila) for fila in datos if fila]

#Vemos la cantidad total de registros del json. 
print("La cantidad total de registros es:", len(registros))

La cantidad total de registros es: 746909


In [3]:
#Función para filtrar solo los DNS.
def fDNS(dictionary, search_string):
    def iterator_func(x):
        if search_string in list(x.keys()):
            return True
        return False
    return list(filter(iterator_func, dictionary))

In [4]:
#Mostramos la nueva cantidad de registros filtrados.
filtrarDNS = fDNS(registros, "dns")
print("La cantidad de registros de DNS es:", len(filtrarDNS))

La cantidad de registros de DNS es: 15749


In [5]:
#Mostramos la información de dos registros cualquiera.
print("Información de dos registros:")
print(filtrarDNS[0:2])

Información de dos registros:
[{'timestamp': '2017-07-22T17:33:16.661646-0500', 'flow_id': 1327836194150542, 'pcap_cnt': 22269, 'event_type': 'dns', 'vlan': 110, 'src_ip': '2001:0dbb:0c18:0011:0260:6eff:fe30:0863', 'src_port': 59680, 'dest_ip': '2001:0500:0001:0000:0000:0000:803f:0235', 'dest_port': 53, 'proto': 'UDP', 'dns': {'type': 'query', 'id': 15529, 'rrname': 'api.wunderground.com', 'rrtype': 'A', 'tx_id': 0}}, {'timestamp': '2017-07-22T17:33:24.990320-0500', 'flow_id': 2022925111925872, 'pcap_cnt': 54352, 'event_type': 'dns', 'vlan': 110, 'src_ip': '2001:0dbb:0c18:0011:0260:6eff:fe30:0863', 'src_port': 38051, 'dest_ip': '2001:0500:0003:0000:0000:0000:0000:0042', 'dest_port': 53, 'proto': 'UDP', 'dns': {'type': 'query', 'id': 58278, 'rrname': 'stork79.dropbox.com', 'rrtype': 'A', 'tx_id': 0}}]


In [7]:
#Volvemos el json a un dataframe.
df = pd.json_normalize(filtrarDNS)
df.head(5)

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN
3,2017-07-22T17:33:27.380146-0500,578544790391795,54520,dns,150,192.168.207.4,53,192.168.205.170,31393,UDP,answer,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,NaN,NaN,NXDOMAIN,NaN,NaN
4,2017-07-22T17:33:27.380146-0500,578544790391795,54520,dns,150,192.168.207.4,53,192.168.205.170,31393,UDP,answer,54724,<root>,SOA,NaN,NXDOMAIN,20864.0,NaN


In [10]:
#Filtramos los registros de tipo A. 
filtrarA = df[df['dns.rrtype'] == 'A']
print("La cantidad de registros de DNA de tipo A es:", len(filtrarA))

La cantidad de registros de DNA de tipo A es: 2849


In [12]:
#Filtramos los dominios únicos
filtrarUnicos = filtrarA.drop_duplicates(subset=['dns.rrname'])
print("Los registros únicos son:", len(filtrarUnicos))

Los registros únicos son: 177


In [14]:
#Obtener el TLD, utilizando flare. 
filtrarUnicos['tld_flare'] = filtrarUnicos['dns.rrname'].apply(domain_tld_extract)

<ipython-input-14-3c768e00d6f5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtrarUnicos['tld_flare'] = filtrarUnicos['dns.rrname'].apply(domain_tld_extract)


In [15]:
filtrarUnicos['tld_flare']
nan_value = float("NaN")
filtrarUnicos.replace("", nan_value, inplace=True)
filtrarUnicos.dropna(subset = ["tld_flare"], inplace=True)

C:\Users\alexa\anaconda3\lib\site-packages\pandas\core\frame.py:4524: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
<ipython-input-15-7154d0e77290>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtrarUnicos.dropna(subset = ["tld_flare"], inplace=True)


In [16]:
#Observamos nuestro nuevo dataset, con los cambios realizados. 
filtrarUnicos.head(5)

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata,tld_flare
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN,wunderground.com
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN,dropbox.com
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN,aoltw.net
6,2017-07-22T17:33:38.537426-0500,2167545251640146,55687,dns,180,192.168.198.62,35092,192.168.207.4,53,UDP,query,7425,safebrowsing.clients.google.com.home,A,0.0,NaN,NaN,NaN,safebrowsing.clients.google.com.home
9,2017-07-22T17:33:45.672579-0500,541745511744323,56307,dns,120,192.168.202.172,55580,192.168.207.4,53,UDP,query,12854,fxfeeds.mozilla.com,A,0.0,NaN,NaN,NaN,mozilla.com


### PARTE 2 - Data Science

In [19]:
#Utilizamos el clasificador del laboratorio pasado. 
dfDS = pd.DataFrame()
datos = filtrarUnicos['tld_flare']
dfDS['longitud'] =  filtrarUnicos['tld_flare'].str.len()
dfDS['digitos'] = filtrarUnicos['tld_flare'].str.count('[0-9]')
dfDS['entropia'] = filtrarUnicos['tld_flare'].apply(H_entropy)
dfDS['proporcionVocalesConsonantes'] = filtrarUnicos['tld_flare'].apply(proporcionVocalesConsonantes)
dfDS['posicionPrimerDigito'] = filtrarUnicos['tld_flare'].apply(posicionPrimerDigito)

#Observamos nuestro nuevo dataset. 
dfDS.head(5)

,longitud,digitos,entropia,proporcionVocalesConsonantes,posicionPrimerDigito
0,16,0,3.375000,0.500000,0
1,11,0,3.027169,0.428571,0
2,9,0,2.947703,0.600000,0
6,36,0,3.861052,0.600000,0
9,11,0,2.913977,0.666667,0


In [22]:
#Cargamos nuestro modelo del laboratorio pasado. 
modelo = load('modelo1.joblib')
resultados = modelo.predict(dfDS)

In [23]:
#Función para filtrar los registros.
registrosFil = []
for i in resultados:
    if i == 0:
        registrosFil.append(False)
    else:
        registrosFil.append(True)

In [37]:
print("Registros filtrados:")
filtrarUnicos[registrosFil]['tld_flare']

Registros filtrados:


6                  safebrowsing.clients.google.com.home
163                            192.168.22.110phpmyadmin
191         safebrowsing.clients.google.com.localdomain
293                192.168.22.110phpmyadmin.localdomain
363                                       ntkrnlpa.info
463                                     portswigger.net
583                                      sql-ledger.org
648                                 backtrack-linux.org
796      safebrowsing.clients.google.com.hackerlabs.vpn
828                                      phpmyadmin.net
887                                      postgresql.org
996              secure.informaction.com.hackerlabs.vpn
1072                safebrowsing.clients.google.com.lan
1195                                   "192.168.206.56"
1398                                  windowsupdate.com
1836                                    192.168.26-27.0
2182                                       fastserv.com
2235                                      liquid

In [35]:
#Implementamos DGA con Flare.
from features import dga_classifier
dga = dga_classifier()

[*] Initializing... training classifier - Please wait.
[+] Training classifier on training set


C:\Users\alexa\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:541: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\alexa\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:545: RuntimeWarning: invalid value encountered in true_divide
  decision = (predictions[k] /


[+] Out of sample legit f1 score 0.9989011490540403
[+] Out of sample dga f1 score 0.6373117033603707
[+] Training final classifier


C:\Users\alexa\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:541: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\alexa\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:545: RuntimeWarning: invalid value encountered in true_divide
  decision = (predictions[k] /


[+] Classifier Ready


In [36]:
#Filtramos los dominios clasificados como DGA.
filtrarUnicos['good'] = filtrarUnicos['tld_flare'].apply(dga.predict)
filtrarUnicos[filtrarUnicos['good'] == 'dga']['tld_flare']

<ipython-input-36-44ede040df75>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtrarUnicos['good'] = filtrarUnicos['tld_flare'].apply(dga.predict)


163      192.168.22.110phpmyadmin
9856         vtlfccmfxlkgifuf.com
10048        1922.168.22.254.home
10646        ejfodfmfxlkgifuf.xyz
14730        192.168.21-28.0.home
Name: tld_flare, dtype: object

**¿Son los mismos dominios que los de su modelo?**  
Si los cinco registros que se obtuvieron al utilizar el modelo de DGA con Flare, se encuentran en la lista de registros filtrados por el modelo del laboratorio pasado. 

### PARTE 3 - Dominio Experto 

In [46]:
from umbrella import *
umbrellaCisco = Umbrella(limit= 1000000)

NameError: name 'Umbrella' is not defined